# Doc for kinds generation

- specific thresholds
- standard thresholds
- example on how properties are changing (ex. classification of magmoms wrt threshold and so on)

# Automatic kinds determination

In the following we will show how it is possible to generate automatically kinds and how to specify a set of property thresholds to help the algorithm find a configuration which satisfies your needs.
We are going to use the mutable version of the `StructureData`, for this example.

NB: the automatic detection of kinds considers only `site` properties (charges, magmoms and so on), not `global` ones (e.g. pbc, hubbard and so on). For now, we exclude the `weights` property.

## Explanation of the classification algorithm

bla bla bla...

### Ideas: what about using a K-means? 

In [1]:
from aiida import load_profile
load_profile()

from aiida_atomistic import StructureDataMutable

structure_dict = {
    'pbc': [True, True, True],
    'cell': [
        [2.8403, 0.0, 1.7391821518091137e-16],
        [-1.7391821518091137e-16, 2.8403, 1.7391821518091137e-16],
        [0.0, 0.0, 2.8403],
        ],
    'symbols': ['Fe', 'Fe'],
    'positions': [[0.0, 0.0, 0.0], [1.42015, 1.42015, 1.4201500000000002]],
    'magmoms': [[2.5, 0.0, 0.0], [2.1, 0.0, 0.0]],
}

structure = StructureDataMutable(**structure_dict)
structure.properties.magmoms

[[2.5, 0.0, 0.0], [2.1, 0.0, 0.0]]

As we can see, this structure contains two Fe atoms with a different magnetic moment (`magmoms` property). 
This means that we can define two different kinds. 
We could do this by hands, just defining

In [2]:
structure.properties.kinds = ['Fe1', 'Fe2']

such that:

In [3]:
structure.properties.sites

[<SiteImmutable: kind name 'Fe1' @ 0.0,0.0,0.0>,
 <SiteImmutable: kind name 'Fe2' @ 1.42015,1.42015,1.4201500000000002>]

or we can do it automatically. This can be done using the `get_kinds` method of the structure. Let's suppose to just generate the kinds, and not to store them in the structure (`ready_to_use=False`, the default):

In [4]:
structure.get_kinds(ready_to_use=False)

{'kinds': ['Fe0', 'Fe1'],
 'masses': array([55.845, 55.845]),
 'charges': array([0., 0.]),
 'magmoms': array([[2.5, 0. , 0. ],
        [2.1, 0. , 0. ]]),
 'symbols': ['Fe', 'Fe'],
 'positions': [[0.0, 0.0, 0.0], [1.42015, 1.42015, 1.4201500000000002]]}

We find indeed two kinds. This are obtained considering a default threshold for the magnetic moments (for each element of the magmom vector):

In [5]:
from aiida_atomistic.data.structure.mixin import _DEFAULT_THRESHOLDS

_DEFAULT_THRESHOLDS

{'charges': 0.1, 'masses': 0.0001, 'magmoms': 0.0001}

we see that for magmoms, the threshold is 1e-4 for each component. 

## Defining custom thresholds for the properties

We can provide a custom threshold to the `get_kinds` method. Let's suppose to impose a large threshold, so that we find only one kind. This can be done easily:

In [6]:
structure.get_kinds(custom_thr={'magmoms': 0.5},ready_to_use=False)

{'kinds': ['Fe0', 'Fe0'],
 'masses': array([55.845, 55.845]),
 'charges': array([0., 0.]),
 'magmoms': array([[2.5, 0. , 0. ],
        [2.5, 0. , 0. ]]),
 'symbols': ['Fe', 'Fe'],
 'positions': [[0.0, 0.0, 0.0], [1.42015, 1.42015, 1.4201500000000002]]}

We find indeed only one kind. Moreover, we see that the magmoms values is now the same, indeed indicating that they are of the same kind. This value is chosen to be the upper value of the given box in the property space.

This works also for multiple properties at the same time. Let's suppose to have also charges:

In [7]:
structure.properties.charges = [+1,-1]

We then compute again the kinds

In [8]:
structure.get_kinds(custom_thr={'magmoms': 0.5},ready_to_use=False)

{'kinds': ['Fe0', 'Fe1'],
 'masses': array([55.845, 55.845]),
 'charges': array([ 1., -1.]),
 'magmoms': array([[2.5, 0. , 0. ],
        [2.5, 0. , 0. ]]),
 'symbols': ['Fe', 'Fe'],
 'positions': [[0.0, 0.0, 0.0], [1.42015, 1.42015, 1.4201500000000002]]}

which is correct: the `magmoms` are found to be the same (withing the large imposed threshold), whereas the `charges` are not (the default value si `0.1`), so we have again two kinds. 
Let's try to play also with the `charges` threshold:

In [9]:
structure.get_kinds(custom_thr={'magmoms': 0.5, 'charges':2.5},ready_to_use=False)

{'kinds': ['Fe0', 'Fe0'],
 'masses': array([55.845, 55.845]),
 'charges': array([1., 1.]),
 'magmoms': array([[2.5, 0. , 0. ],
        [2.5, 0. , 0. ]]),
 'symbols': ['Fe', 'Fe'],
 'positions': [[0.0, 0.0, 0.0], [1.42015, 1.42015, 1.4201500000000002]]}

again, one kind.

## Excluding some properties in the kinds determination

It is also possible to ignore some property in the kinds determination. This should be used with particular attention, as using directly its results can lead to inconsistencies in calculations. To exclude a property in the analysis, we can just put it in the `exclude` input `list`:

In [10]:
structure.get_kinds(exclude=["magmoms"],ready_to_use=False)

{'kinds': ['Fe0', 'Fe1'],
 'masses': array([55.845, 55.845]),
 'charges': array([ 1., -1.]),
 'symbols': ['Fe', 'Fe'],
 'positions': [[0.0, 0.0, 0.0], [1.42015, 1.42015, 1.4201500000000002]]}

In [11]:
structure.get_kinds(exclude=["magmoms"])

{'kinds': ['Fe0', 'Fe1'],
 'masses': array([55.845, 55.845]),
 'charges': array([ 1., -1.]),
 'symbols': ['Fe', 'Fe'],
 'positions': [[0.0, 0.0, 0.0], [1.42015, 1.42015, 1.4201500000000002]]}

## Automatically updating the kinds for a given (mutable) `StructureData`

We provide a method, the `set_automatic_kinds`, which allows you to basically update a structure by means of the automatic kinds generation:

In [15]:
structure.set_automatic_kinds(exclude=[],custom_thr={})
structure.properties.kinds

['Fe0', 'Fe1']